In [11]:
import os
import numpy as np
from PIL import Image
import random
import math
import pandas as pd

In [ ]:
# 1. Carica l’immagine e normalizza in [0,1]
img = Image.open('cat.jpg').convert('RGB')
img_arr = np.asarray(img, dtype=np.float32) / 255.0
noisy_arr = add_salt_pepper_noise(img_arr, amount=0.1)
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))

# 4a. Salva su disco
noisy_img.save('noisy_image.jpg')

# 4b. Oppure mostra a video (apre il viewer predefinito)
noisy_img.show()


FileNotFoundError: [Errno 2] No such file or directory: 'cat.jpg'

In [6]:
def add_salt_pepper_noise(image: np.ndarray, amount: float = 0.05, salt_vs_pepper: float = 0.5) -> np.ndarray:
    """
    :param image: Input image as float32 numpy array in [0,1], shape (H, W, 3).
    :param amount: Fraction of total pixels to corrupt.
    :param salt_vs_pepper: Fraction of corrupted pixels that are salt (vs pepper).
    :return: Noisy image as numpy array.
    """
    noisy = np.copy(image)
    ROWS, COLS, _ = image.shape
    num_pixels = ROWS * COLS
    num_salt = int(np.ceil(amount * num_pixels * salt_vs_pepper))
    num_pepper = int(np.ceil(amount * num_pixels * (1 - salt_vs_pepper)))

    # SALT
    coords_y = np.random.randint(0, ROWS, num_salt)
    coords_x = np.random.randint(0, COLS, num_salt)
    noisy[coords_y, coords_x, :] = 1.0

    # PEPPER
    coords_y = np.random.randint(0, ROWS, num_pepper)
    coords_x = np.random.randint(0, COLS, num_pepper)
    noisy[coords_y, coords_x, :] = 0.0

    return noisy



def add_gaussian_noise(image: np.ndarray, mean: float = 0.0, var: float = 0.01) -> np.ndarray:
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param mean: Mean of Gaussian noise.
    :param var: Variance of Gaussian noise.
    :return: Noisy image as numpy array.
    """
    sigma = math.sqrt(var)
    gauss = np.random.normal(mean, sigma, image.shape)
    noisy = image + gauss
    return np.clip(noisy, 0, 1)



def add_uniform_noise(image: np.ndarray, low: float = -0.05, high: float = 0.05) -> np.ndarray:
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param low: Lower bound of uniform noise.
    :param high: Upper bound of uniform noise.
    :return: Noisy image as numpy array.
    """
    uniform_noise = np.random.uniform(low, high, image.shape)
    noisy = image + uniform_noise
    return np.clip(noisy, 0, 1)

In [12]:
def generate_dataset(input_dir: str, output_dir: str) -> None:
    # Noise specifications
    specs = {
        'original': None,
        'salt_pepper': {'func': add_salt_pepper_noise, 'params': {'amount': 0.05, 'salt_vs_pepper': 0.5}},
        'gaussian': {'func': add_gaussian_noise, 'params': {'mean': 0.0, 'var': 0.01}},
        'uniform': {'func': add_uniform_noise, 'params': {'low': -0.05, 'high': 0.05}}
    }
    
    # Mapping folder names to labels
    label_map = {
        'original': 'Original',
        'salt_pepper': 'Salt & Pepper',
        'gaussian': 'Gaussian',
        'uniform': 'Uniform'
    }

    # Create output subdirectories
    for folder in specs:
        os.makedirs(os.path.join(output_dir, folder), exist_ok=True)

    # Prepare records for supervised labels
    records = []

    # Process images
    for fname in os.listdir(input_dir):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue
        # Load and normalize
        img = Image.open(os.path.join(input_dir, fname)).convert('RGB')
        arr = np.asarray(img, dtype=np.float32) / 255.0
        
        # Save original
        save_path = os.path.join(output_dir, 'original', fname)
        img.save(save_path)
        records.append({'filepath': save_path, 'label': label_map['original']})
        
        # Apply noise types
        for folder, spec in specs.items():
            if spec is None:
                continue
            noisy_arr = spec['func'](arr, **spec['params'])
            noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
            noisy_path = os.path.join(output_dir, folder, fname)
            noisy_img.save(noisy_path)
            records.append({'filepath': noisy_path, 'label': label_map[folder]})
    
    # Save labels CSV
    df = pd.DataFrame(records)
    csv_path = os.path.join(output_dir, 'labels.csv')
    df.to_csv(csv_path, index=False)
    print(f"Dataset and labels saved. CSV written to: {csv_path}")

In [13]:
# ESEGUO
input_directory = "."
output_directory = "dataset/."
generate_dataset(input_directory, output_directory)

Dataset and labels saved. CSV written to: dataset/.\labels.csv
